In [ ]:
# import sys
# sys.path.insert(1, '/path/to/CGCompiler/')

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches

import seaborn as sns

from user.usersettings import feasible_bead_types

from user.analysis.utils import read_distribution_file

from user.usersettings import training_systems, bonds_to_optimize, angles_to_optimize
from user.usersettings import target_dict, X_cont_upperbounds

SMALL_SIZE = 16
MEDIUM_SIZE = 20 
BIGGER_SIZE = 24

plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rc('axes', titlesize=SMALL_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rcParams['legend.fontsize'] = SMALL_SIZE

In [ ]:
plt.rcParams['figure.figsize'] = 16, 12
plt.rcParams['legend.fontsize'] = SMALL_SIZE

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
def read_df_list(nsim):
    df_list = []
    for i in range(nsim):
        df_list.append(pd.read_csv("output/population-%d.csv" %(i), index_col=[0,1,2],
                                   header=[0,1], skipinitialspace=True))
        
    return df_list

In [ ]:
niter = 10
df = pd.concat(read_df_list(niter))
df = df.reorder_levels([2,0,1])
df

In [ ]:
df.sort_index(level=[0,1], inplace=True)

In [ ]:
df.loc[(slice(0), slice(None), slice(None))]

In [ ]:
fitness_ndx1 = "Unnamed: 37_level_1"
fitness_ndx_tup = ("fitness", "Unnamed: 37_level_1")

df_sorted = df.sort_values(("fitness", fitness_ndx1))

In [ ]:
iter_grps = df.groupby('iteration')

In [ ]:
nbest = 4

tuples = []
for iter_ndx in range(niter):
    #print(iter_ndx)
    for rank in range(nbest):
        #print((iter_ndx, rank))
        tuples.append((iter_ndx, rank))
index_temp = pd.MultiIndex.from_tuples(tuples, names=['iteration', 'rank'])

In [ ]:

fig, ax = plt.subplots(1, figsize=(9,6))


df_temp = pd.DataFrame(
    np.zeros(nbest*niter),
    dtype='float',
    index=index_temp,
    columns=['fitness']
)


for iter_ndx in range(niter):  
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(("fitness", fitness_ndx1))[:nbest]
    
    df_temp.loc[(iter_ndx, slice(None)), slice(None)] = df_i_sorted["fitness", fitness_ndx1].values
    xvals = np.zeros(nbest) + iter_ndx
    ax.plot(xvals, df_i_sorted["fitness", fitness_ndx1], 'o', color='C0')
    
ax.set_xlabel('iteration')
ax.set_ylabel('cost')



In [ ]:
iterrange = np.arange(niter)
means = np.zeros(niter)
stds = np.zeros(niter)
mins = np.zeros(niter)
maxs = np.zeros(niter)
#gbest_iters = 

for i in range(niter):
    means[i] = df.xs(i, level='iteration').loc[(slice(None), slice(None), slice(None)),"fitness"].mean()
    stds[i] = df.xs(i, level='iteration').loc[(slice(None), slice(None), slice(None)),"fitness"].std(ddof=1)
    mins[i] = df.xs(i, level='iteration').loc[(slice(None), slice(None), slice(None)),"fitness"].min()
    maxs[i] = df.xs(i, level='iteration').loc[(slice(None), slice(None), slice(None)),"fitness"].max()

plt.figure(figsize=(9,6))
    
plt.plot(iterrange, means, 'o-', color='C0', label='mean')
plt.fill_between(iterrange, mins, maxs, alpha=0.2, color='grey', label='range')
plt.fill_between(iterrange, means - stds, means + stds, alpha=0.2, color='C0', label='mean + std')



for iter_ndx in range(niter):
    #print(iter_ndx, grp)
    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    
    df_temp.loc[(iter_ndx, slice(None)), slice(None)] = df_i_sorted["fitness", fitness_ndx1].values
    #print(grp_sorted)
    xvals = np.zeros(nbest) + iter_ndx
    #plt.plot(xvals, df_i_sorted["fitness", "Unnamed: 37_level_1"], 'o', color='C1', alpha=1)
    #ax[1].plot(xvals, df_i_sorted[bond, 'fc'], 'o', color='C0')

    
rank_grps = df_temp.groupby('rank')
for rank, grp in rank_grps:
    alpha = 1 - rank * 1 / nbest
    #print(alpha)
    if rank == 0:
        plt.plot(iterrange, grp['fitness'], 'o', color='C1',
                 alpha=alpha,
                 label='g%dbest(i)' %nbest
                )
    else:
        plt.plot(iterrange, grp['fitness'], 'o', color='C1',
         alpha=alpha,
         #label='%d best solutions' %nbest
        )

plt.xlabel('iteration')
plt.ylabel('cost / a.u.')
plt.xticks(iterrange[::2])
#plt.grid()
plt.legend()
plt.ylim(9,101)

plt.tight_layout()
plt.show()

In [ ]:
df_sns = df.reset_index()
df_sns2 = pd.DataFrame(
    data=np.column_stack([df_sns['iteration'].values, df_sns['fitness', fitness_ndx1]]),
    index=df_sns.index,
    columns=['iteration', 'fitness']
)
df_sns2

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

sns.boxplot(x='iteration', y='fitness', data=df_sns2, ax=ax)

ax.plot(iterrange, rank_grps.get_group(0), 'o-', color='C1', label='gbest(t)')

ax.set_xticks(ticks=np.arange(0,niter, 2), labels=np.arange(0, niter, 2))

ax.set_ylabel('cost')

ax.set_xlim(-0.5, 40.5)
ax.set_ylim(10, 90)

plt.legend()
plt.tight_layout()

plt.show()

In [ ]:


fig, ax = plt.subplots(2)

for iter_ndx in range(niter):
    #print(iter_ndx, grp)
    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    #print(grp_sorted)
    xvals = np.zeros(nbest) + iter_ndx
    ax[0].plot(xvals, df_i_sorted['AM1', "Unnamed: 35_level_1"], 'o', color='C0')
    ax[1].plot(xvals, df_i_sorted['AM2', "Unnamed: 36_level_1"], 'o', color='C0')
    
ax[1].set_xlabel('iteration')
ax[0].set_ylabel('AM1 beadtype')
ax[1].set_ylabel('AM2 beadtype')

plt.show()

In [ ]:
#nbest = 10

bond = 'PO4-AM1'

fig, ax = plt.subplots(2)

for iter_ndx in range(niter):    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    xvals = np.zeros(nbest) + iter_ndx
    ax[0].plot(xvals, df_i_sorted[bond, 'b0'], 'o', color='C0')
    ax[1].plot(xvals, df_i_sorted[bond, 'fc'], 'o', color='C0')
    
ax[1].set_xlabel('iteration')
ax[0].set_ylabel('b0 / nm')
ax[1].set_ylabel(r'fc / kJ/mol/nm$^2$')

plt.show()

In [ ]:
bond = 'AM1-AM2'

fig, ax = plt.subplots(2)

for iter_ndx in range(niter):    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    xvals = np.zeros(nbest) + iter_ndx
    ax[0].plot(xvals, df_i_sorted[bond, 'b0'], 'o', color='C0')
    ax[1].plot(xvals, df_i_sorted[bond, 'fc'], 'o', color='C0')
    
ax[1].set_xlabel('iteration')
ax[0].set_ylabel('b0 / nm')
ax[1].set_ylabel(r'fc / kJ/mol/nm$^2$')

plt.show()

In [ ]:
bond = 'AM1-T1A'

fig, ax = plt.subplots(2)

for iter_ndx in range(niter):    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    xvals = np.zeros(nbest) + iter_ndx
    ax[0].plot(xvals, df_i_sorted[bond, 'b0'], 'o', color='C0')
    ax[1].plot(xvals, df_i_sorted[bond, 'fc'], 'o', color='C0')
    
ax[1].set_xlabel('iteration')
ax[0].set_ylabel('b0 / nm')
ax[1].set_ylabel(r'fc / kJ/mol/nm$^2$')

plt.show()

In [ ]:
bond = 'AM2-C1B'

fig, ax = plt.subplots(2)

for iter_ndx in range(niter):    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    xvals = np.zeros(nbest) + iter_ndx
    ax[0].plot(xvals, df_i_sorted[bond, 'b0'], 'o', color='C0')
    ax[1].plot(xvals, df_i_sorted[bond, 'fc'], 'o', color='C0')
    
ax[1].set_xlabel('iteration')
ax[0].set_ylabel('b0 / nm')
ax[1].set_ylabel(r'fc / kJ/mol/nm$^2$')

plt.show()

In [ ]:
bond = 'PO4-AM1-AM2'

fig, ax = plt.subplots(2)

for iter_ndx in range(niter):
    #print(iter_ndx, grp)
    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    #print(grp_sorted)
    xvals = np.zeros(nbest) + iter_ndx
    ax[0].plot(xvals, df_i_sorted[bond, 'b0'], 'o', color='C0')
    ax[1].plot(xvals, df_i_sorted[bond, 'fc'], 'o', color='C0')
    
ax[1].set_xlabel('iteration')
ax[0].set_ylabel('a0 / nm')
ax[1].set_ylabel(r'fc / kJ/mol/deg$^2$')

plt.show()

In [ ]:
bond = 'PO4-AM1-T1A'

fig, ax = plt.subplots(2)

for iter_ndx in range(niter):
    #print(iter_ndx, grp)
    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    #print(grp_sorted)
    xvals = np.zeros(nbest) + iter_ndx
    ax[0].plot(xvals, df_i_sorted[bond, 'b0'], 'o', color='C0')
    ax[1].plot(xvals, df_i_sorted[bond, 'fc'], 'o', color='C0')
        
ax[1].set_xlabel('iteration')
ax[0].set_ylabel('a0 / nm')
ax[1].set_ylabel(r'fc / kJ/mol/deg$^2$')

plt.show()

In [ ]:
bond = 'AM1-T1A-C2A'

fig, ax = plt.subplots(2)

for iter_ndx in range(niter):
    #print(iter_ndx, grp)
    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    #print(grp_sorted)
    xvals = np.zeros(nbest) + iter_ndx
    ax[0].plot(xvals, df_i_sorted[bond, 'b0'], 'o', color='C0')
    ax[1].plot(xvals, df_i_sorted[bond, 'fc'], 'o', color='C0')
    
ax[1].set_xlabel('iteration')
ax[0].set_ylabel('a0 / nm')
ax[1].set_ylabel(r'fc / kJ/mol/deg$^2$')

plt.show()

In [ ]:
bond = 'AM2-C1B-C2B'

fig, ax = plt.subplots(2)

for iter_ndx in range(niter):
    #print(iter_ndx, grp)
    
    df_i_sorted = df.loc[(slice(iter_ndx), slice(None), slice(None))].sort_values(fitness_ndx_tup)[:nbest]
    #print(grp_sorted)
    xvals = np.zeros(nbest) + iter_ndx
    ax[0].plot(xvals, df_i_sorted[bond, 'b0'], 'o', color='C0')
    ax[1].plot(xvals, df_i_sorted[bond, 'fc'], 'o', color='C0')
    
ax[1].set_xlabel('iteration')
ax[0].set_ylabel('a0 / nm')
ax[1].set_ylabel(r'fc / kJ/mol/deg$^2$')

plt.show()

In [ ]:
def load_obs_dict(uid, outputdir, training_systems):
    
    obs_dict = {}
    for molkey in training_systems:
        obs_dict[molkey] = {}
        for tr_system in training_systems[molkey]:
                path = os.path.join(outputdir, uid, molkey, tr_system, 'observables_dict.dat')

                obs_dict[molkey][tr_system] = json.load(open(path, 'r'))
            
    return obs_dict

def nbest_obs_dict(df, outputdir, training_systems, nbest):
    obs_dict_dict = {}
    for i in range(nbest):
        uid = df.index.get_level_values(level='uid')[i]
        obs_dict_dict[i] = load_obs_dict(uid, outputdir, training_systems)
    #uid = uids[0]
    
    return obs_dict_dict

In [ ]:
nbest_obsdict_total = nbest_obs_dict(df_sorted, 'output/', training_systems, 10)

In [ ]:
nbest_obsdict_total

In [ ]:
molkey = 'DPSM'

fig, ax = plt.subplots(len(bonds_to_optimize[molkey]), 1, figsize=(12,12))


    
tr_system = 'DPSM128_328K'
ax[0,].set_title(tr_system)
ax[-1,].set_xlabel('bond length / nm')
ax[-2].set_ylabel('probability a.u.')

for i in range(10):
    for j, bond in enumerate(bonds_to_optimize[molkey]):
        if i == 0:
            lw = 3
            color = 'C1'
            zorder = 2.1
            label='gbest'
        else:
            lw = 2
            color = 'grey'
            zorder = 2
            label=None
        dist, bins = nbest_obsdict_total[i][molkey][tr_system]['bond_lengths_dist'][bond]
        ax[j].plot(bins[1:],dist, lw=lw, color=color, zorder=zorder, label=label)

        
        if i == 0:
            target_dist, target_bins = target_dict[molkey]['bond_lengths_dist'][molkey][tr_system][bond]
            ax[j].plot(target_bins[1:], target_dist, color='firebrick', label='target CHARMM SSM', lw=3,
                      zorder=2.01)
            ax[j].text(0.2,0.15, bond, fontweight="bold", fontsize=14,ha="center", transform=ax[j].transAxes)
            
        
ax[1].legend()

plt.show()

In [ ]:
molkey = 'DPSM'

fig, ax = plt.subplots(len(angles_to_optimize[molkey]), 1, figsize=(12,12))


    
tr_system = 'DPSM128_328K'
ax[0,].set_title(tr_system)
ax[-1,].set_xlabel('angle / deg')
ax[-2].set_ylabel('probability a.u.')

for i in range(10):
    for j, bond in enumerate(angles_to_optimize[molkey]):
        if i == 0:
            lw = 2
            color = 'C1'
            zorder = 2.1
            label='gbest'
        else:
            lw = 2
            color = 'grey'
            zorder = 2
            label=None
        
        dist, bins = nbest_obsdict_total[i][molkey][tr_system]['angles_dist'][bond]
        ax[j].plot(bins[1:],dist, lw=lw, color=color, zorder=zorder, label=label)
        if i == 0:
            target_dist, target_bins = target_dict[molkey]['angles_dist'][molkey][tr_system][bond]
            ax[j].plot(target_bins[1:], target_dist, color='firebrick', label='target CHARMM SSM', lw=3,
                      zorder=2.01)
            ax[j].text(0.2,0.15, bond, fontweight="bold", fontsize=14,ha="center", transform=ax[j].transAxes)

            
ax[1].legend()

plt.show()